In [ ]:
import tkinter as tk
from tkinter import ttk
import joblib
import pandas as pd

# Load the ensemble model and encoders
ensemble_model = joblib.load('ensemble_model.joblib')
label_encoder_country = joblib.load('label_encoder_country.joblib')
label_encoder_city = joblib.load('label_encoder_city.joblib')
ordinal_encoder_time_slot = joblib.load('ordinal_encoder_time_slot.joblib')

# Define time slots
def get_detailed_time_slot(hour):
    if 0 <= hour < 6:
        return 'Late Night'
    elif 6 <= hour < 9:
        return 'Early Morning'
    elif 9 <= hour < 12:
        return 'Late Morning'
    elif 12 <= hour < 15:
        return 'Early Afternoon'
    elif 15 <= hour < 18:
        return 'Late Afternoon'
    elif 18 <= hour < 21:
        return 'Early Evening'
    else:
        return 'Night'


# Prediction function
def predict_outage(country_input, city_input, time_slot_input):
    try:
        # Encode the country and city inputs
        country_encoded = label_encoder_country.transform([country_input])[0]
        city_encoded = label_encoder_city.transform([city_input])[0]
    except ValueError:
        return "Country or City not recognized"

    # Convert time_slot_input to hour and then to the corresponding time slot category
    time_slot_to_hour = {'Late Night': 0, 'Early Morning': 6, 'Late Morning': 9, 
                         'Early Afternoon': 12, 'Late Afternoon': 15, 
                         'Early Evening': 18, 'Night': 21}
    hour = time_slot_to_hour.get(time_slot_input, -1)

    if hour == -1:
        return "Time slot not recognized"

    time_slot_category = get_detailed_time_slot(hour)

    try:
        time_slot_encoded = ordinal_encoder_time_slot.transform([[time_slot_category]])[0][0]
    except ValueError:
        return "Time slot category not recognized"

    # Default values for other features (replace with actual values as per your notebook)
    default_values = {
        'proportion_timeout': 1.234079246734771,  # replace with the average or typical value if known
        'proportion_unreachable': 1.12382143954133,
        'proportion_terminated': 1.54313618470646,
        'avg_dns_success_time': 1.123814659476422,
        'avg_dns_failure_time': 1.432988650019605,
        'count_dns_failure': 1.125727213548191,
        'ssl_error_prop': 1.87639002049091
    }
    
    # Create a DataFrame for all input features
    input_features = pd.DataFrame({
        'country_encoded': [country_encoded],
        'city_encoded': [city_encoded],
        'time_slot_encoded': [time_slot_encoded],
        **default_values
    })

    # Make a prediction using the ensemble model
    numeric_prediction = ensemble_model.predict(input_features)[0]
    
    # Mapping from integer to string labels
    reverse_label_map = {0: 'good', 1: 'moderate', 2: 'bad', 3: 'worse'}
    string_prediction = reverse_label_map.get(numeric_prediction, "Unknown")

    return string_prediction

# Function to be called when the Predict button is clicked
def on_predict():
    country = country_entry.get()
    city = city_entry.get()
    time_slot = time_slot_entry.get()

    prediction = predict_outage(country, city, time_slot)
    result_label.config(text=f'Prediction: {prediction}')

# Setting up the GUI
root = tk.Tk()
root.title("Outage Prediction App")

# Creating layout
ttk.Label(root, text="Country:").grid(row=0, column=0, padx=10, pady=10)
country_entry = ttk.Entry(root)
country_entry.grid(row=0, column=1, padx=10, pady=10)

ttk.Label(root, text="City:").grid(row=1, column=0, padx=10, pady=10)
city_entry = ttk.Entry(root)
city_entry.grid(row=1, column=1, padx=10, pady=10)

ttk.Label(root, text="Time Slot:").grid(row=2, column=0, padx=10, pady=10)
time_slot_entry = ttk.Entry(root)
time_slot_entry.grid(row=2, column=1, padx=10, pady=10)

predict_button = ttk.Button(root, text="Predict", command=on_predict)
predict_button.grid(row=3, column=0, columnspan=2, pady=10)

result_label = ttk.Label(root, text="Prediction: ")
result_label.grid(row=4, column=0, columnspan=2)

root.mainloop()
